# 01 — PLSR Baseline (Soil Spectroscopy)
Load data → preprocess (SNV + SG) → PLSR → metrics (R², RMSE, MAE, RPD) → parity plot.


In [ ]:

import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.cross_decomposition import PLSRegression
from scipy.signal import savgol_filter
from pathlib import Path

DATA_PATH = Path("../data/soil_spectra_teaching.csv")
TARGET = "SOC"  # change if needed

df = pd.read_csv(DATA_PATH)
display(df.head())



# Detect spectral columns (prefix 'wl_') and target
spec_cols = [c for c in df.columns if str(c).startswith("wl_")]
assert len(spec_cols) > 10, "Need >10 spectral columns named like wl_400, wl_402, ..."
X = df[spec_cols].to_numpy(dtype=float)
y = df[TARGET].to_numpy(dtype=float)
print(f"Spectral bands detected: {len(spec_cols)}")



# Preprocessing: SNV + Savitzky–Golay 1st derivative
def snv(mat):
    return (mat - mat.mean(axis=1, keepdims=True)) / (mat.std(axis=1, keepdims=True) + 1e-12)

from math import floor
X_snv = snv(X)
window = min(21, max(5, (X_snv.shape[1]//20)*2+1))  # odd window <=21
X_sg = savgol_filter(X_snv, window_length=window, polyorder=2, deriv=1, axis=1)



# Split
X_train, X_test, y_train, y_test = train_test_split(X_sg, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

# Hyperparameter search
max_comp = min(25, X_train_sc.shape[1]//5 if X_train_sc.shape[1] >= 10 else 5)
param_grid = {"n_components": list(range(2, max(3, max_comp)))}
pls = PLSRegression(scale=False)
gcv = GridSearchCV(pls, param_grid, scoring="neg_mean_squared_error", cv=5, n_jobs=-1)
gcv.fit(X_train_sc, y_train)
best = gcv.best_estimator_
print("Best params:", gcv.best_params_)



# Evaluate
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
y_pred = best.predict(X_test_sc).ravel()
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)

def rpd(y_true, y_hat):
    sd = np.std(y_true, ddof=1)
    rmse_ = mean_squared_error(y_true, y_hat, squared=False)
    return sd / (rmse_ + 1e-12)

print({"R2": r2, "RMSE": rmse, "MAE": mae, "RPD": rpd(y_test, y_pred)})



# Parity plot
plt.figure(figsize=(5,5))
plt.scatter(y_test, y_pred, alpha=0.6, edgecolor='none')
mn, mx = min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())
plt.plot([mn, mx], [mn, mx], linestyle="--")
plt.xlabel(f"Observed {TARGET}")
plt.ylabel(f"Predicted {TARGET}")
plt.title(f"PLSR Parity Plot — {TARGET} (R2={r2:.2f})")
plt.tight_layout()
plt.show()
